<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/152_langchain_day_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# Load your API key
load_dotenv("/Users/micahshull/Documents/AI_Agents/LangChain/LC_setup_day_00/API_KEYS.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Create the language model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create ONE simple tool
def calculator(expression):
    """Does basic math"""
    try:
        return str(eval(expression))
    except:
        return "Math error"

tools = [
    Tool(
        name="calculator",
        func=calculator,
        description="Does basic math like 2+2 or 10*5"
    )
]

# Use the standard ReAct prompt from LangChain hub (this has all required variables)
prompt = hub.pull("hwchase17/react")

# Create the agent
agent = create_react_agent(llm, tools, prompt)

# Create executor (this runs the agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Test it!
print("=== Testing Basic Agent ===")
response = agent_executor.invoke({"input": "What is 25 + 17?"})
print(f"\nFinal Answer: {response['output']}")

=== Testing Basic Agent ===


> Entering new AgentExecutor chain...
To find the sum of 25 and 17, I will perform the addition operation.  
Action: calculator  
Action Input: 25 + 17  42I now know the final answer  
Final Answer: 42

> Finished chain.

Final Answer: 42



## What You Just Built vs. Regular Python

**With regular Python**, you'd have to write something like:

```python
def basic_agent(question):
    # You'd manually code all the logic:
    if "math" in question or "+" in question:
        # Extract numbers somehow
        # Call calculator
        # Format response
    elif "weather" in question:
        # Call weather API
        # Format response
    # etc...
```

**With LangChain**, the agent automatically:
- **Reasons** about what tool to use
- **Calls** the right tool
- **Handles** the response
- **Continues** thinking if needed

## The Key LangChain Benefits

### 1. **Automatic Reasoning (ReAct Pattern)**
Your agent follows the **ReAct** pattern: **Reasoning** → **Acting** → **Observing**

Notice in your output:
```
Action: calculator          ← Agent decided what to do
Action Input: 25 + 17      ← Agent formatted the input
42                         ← Tool gave result
Final Answer: 42           ← Agent synthesized final response
```

The LLM automatically figured out it needed math, chose the calculator, and formatted everything.

### 2. **Tool Abstraction**
```python
tools = [Tool(name="calculator", func=calculator, description="...")]
```

LangChain standardizes how tools work. You just define:
- **What** it does (function)
- **When** to use it (description)
- **What** to call it (name)

The agent reads descriptions and picks tools automatically!

### 3. **Prompt Engineering Built-In**
That `hub.pull("hwchase17/react")` prompt contains sophisticated instructions like:
- How to format tool calls
- When to stop thinking
- How to handle errors
- Multi-step reasoning patterns

You get years of prompt engineering research for free.

### 4. **Error Handling & Parsing**
LangChain handles:
- Malformed tool calls
- Parsing LLM responses
- Tool execution errors
- Infinite loops (max_iterations)

## Why Choose LangChain Over Raw Python?

**Scalability**: Add new tools easily - just append to the list
**Flexibility**: The same agent works with any LLM (OpenAI, Claude, etc.)
**Reliability**: Built-in error handling and retry logic
**Composability**: Chain multiple agents together
**Community**: Thousands of pre-built tools and integrations

## What Makes Your Agent "Smart"

Your agent isn't just following if/else statements. It's:
1. **Reading** your question
2. **Understanding** it needs math
3. **Choosing** the calculator tool
4. **Executing** the tool correctly
5. **Interpreting** the result
6. **Responding** naturally

Try asking it: *"What's 10 * 5 and also tell me what you think about pizza"*

It should use the calculator for math but answer the pizza question directly without tools!

**Want to see the magic?** Try these questions:
- "What's 100 divided by 4?"
- "Tell me a joke" (no tools needed)
- "What's 2+2 and what's the capital of France?"

Each response will show you how the agent *thinks* and *decides* what to do. That's the LangChain advantage - intelligent, automatic decision-making rather than hardcoded logic!

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# Setup (same as before)
load_dotenv("/Users/micahshull/Documents/AI_Agents/LangChain/LC_setup_day_00/API_KEYS.env")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Let's create multiple tools to see the reasoning in action
def calculator(expression):
    """Does basic math calculations"""
    try:
        return str(eval(expression))
    except:
        return "Math error"

def text_analyzer(text):
    """Analyzes text and returns word count and character count"""
    words = len(text.split())
    chars = len(text)
    return f"Words: {words}, Characters: {chars}"

def weather_checker(location):
    """Pretend weather tool (returns fake weather)"""
    return f"The weather in {location} is sunny and 75°F"

tools = [
    Tool(name="calculator", func=calculator, description="Use for math problems like 2+2, 10*5, etc."),
    Tool(name="text_analyzer", func=text_analyzer, description="Use to analyze text - counts words and characters"),
    Tool(name="weather_checker", func=weather_checker, description="Use to check weather for any location")
]

# Get the ReAct prompt to see what's actually sent to the LLM
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=5)

print("="*80)
print("EXAMPLE 1: Single Tool Usage")
print("="*80)
print("Question: What is 15 * 8?")
print()

response1 = agent_executor.invoke({"input": "What is 15 * 8?"})

print("\n" + "="*80)
print("EXAMPLE 2: Multiple Tool Usage")
print("="*80)
print("Question: What's 25 + 30, and how many words are in 'Hello beautiful world'?")
print()

response2 = agent_executor.invoke({"input": "What's 25 + 30, and how many words are in 'Hello beautiful world'?"})

print("\n" + "="*80)
print("EXAMPLE 3: No Tools Needed")
print("="*80)
print("Question: What is the capital of France?")
print()


EXAMPLE 1: Single Tool Usage
Question: What is 15 * 8?



> Entering new AgentExecutor chain...
I need to calculate the product of 15 and 8.  
Action: calculator  
Action Input: 15 * 8  120I now know the final answer  
Final Answer: 120

> Finished chain.

EXAMPLE 2: Multiple Tool Usage
Question: What's 25 + 30, and how many words are in 'Hello beautiful world'?



> Entering new AgentExecutor chain...
I need to perform two tasks: first, calculate the sum of 25 and 30, and second, analyze the text 'Hello beautiful world' to count the words. 

Action: calculator  
Action Input: 25 + 30  55I have the result for the first part of the question, which is 55. Now, I need to analyze the text to count the words.

Action: text_analyzer  
Action Input: 'Hello beautiful world'  Words: 3, Characters: 23I now have the results for both parts of the question. The sum of 25 and 30 is 55, and the text 'Hello beautiful world' contains 3 words. 

Final Answer: The sum of 25 and 30 is 55, and there are 3 

Let me break down exactly how LangChain achieves this "magic" with concrete examples.Now let me explain what's happening behind the scenes:

## 1. **How "Reasoning" Actually Works**

When you ask "What is 15 * 8?", here's what LangChain does:

```python
# LangChain creates this prompt behind the scenes:
full_prompt = f"""
You have access to these tools:
- calculator: Use for math problems like 2+2, 10*5, etc.
- text_analyzer: Use to analyze text - counts words and characters  
- weather_checker: Use to check weather for any location

Use this format:
Question: What is 15 * 8?
Thought: I need to do multiplication, so I should use the calculator tool
Action: calculator
Action Input: 15 * 8
Observation: [result will go here]
Thought: I now have the answer
Final Answer: [final answer here]

Question: What is 15 * 8?
Thought:"""
```

The LLM reads this and generates: "I need to do multiplication, so I should use the calculator tool"

## 2. **How "Tool Calling" Actually Works**

LangChain parses the LLM's response looking for:
- `Action: calculator` ← Which tool to use
- `Action Input: 15 * 8` ← What to pass to the tool

Then it literally runs: `calculator("15 * 8")`

## 3. **How "Response Handling" Works**

```python
# After tool execution, LangChain adds the result back to the prompt:
updated_prompt = previous_prompt + f"""
Observation: 120
Thought: """
```

Then asks the LLM to continue thinking!

## 4. **How "Continued Thinking" Works**

For complex questions, the agent loops:

```
Question: What's 25 + 30, and how many words are in 'Hello beautiful world'?
Thought: I need to do two things - math and text analysis
Action: calculator  
Action Input: 25 + 30
Observation: 55
Thought: Now I need to analyze the text
Action: text_analyzer
Action Input: Hello beautiful world  
Observation: Words: 3, Characters: 19
Thought: I now have both answers
Final Answer: 25 + 30 = 55, and "Hello beautiful world" has 3 words
```

## The Key Insight

**LangChain doesn't have magic AI**. It's essentially:

1. **Smart prompting** - Teaches the LLM a specific format
2. **Response parsing** - Extracts Action/Action Input from LLM responses  
3. **Tool execution** - Runs the actual Python function
4. **Loop management** - Continues until "Final Answer" appears

Run the code above and watch the verbose output. You'll see this exact pattern happening in real-time!

**The "intelligence" comes from**:
- The LLM understanding tool descriptions
- The ReAct prompt format
- LangChain's parsing and execution loop

It's prompt engineering + execution framework, not magic! 🎯

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# Setup
load_dotenv("/Users/micahshull/Documents/AI_Agents/LangChain/LC_setup_day_00/API_KEYS.env")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Simple tool
def calculator(expression):
    return str(eval(expression))

tools = [
    Tool(name="calculator", func=calculator, description="Use for math problems like 2+2, 10*5, etc.")
]

# Let's see what LangChain is actually doing
print("="*80)
print("WHAT LANGCHAIN HIDES FROM YOU")
print("="*80)

# 1. Get the mysterious ReAct prompt
prompt = hub.pull("hwchase17/react")
print("1. The ReAct prompt template:")
print("-" * 40)
print(prompt.template)
print()

# 2. See how LangChain fills in the template
print("2. After LangChain fills in YOUR tools:")
print("-" * 40)
# This is what LangChain does internally
filled_prompt = prompt.partial(
    tools="calculator: Use for math problems like 2+2, 10*5, etc.",
    tool_names="calculator"
)
print(filled_prompt.template)
print()

# 3. See the ACTUAL prompt sent to the LLM
print("3. The ACTUAL prompt sent to OpenAI when you ask 'What is 5*5?':")
print("-" * 40)
final_prompt = filled_prompt.format(
    input="What is 5*5?",
    agent_scratchpad=""  # This gets filled as the agent thinks
)
print(final_prompt)
print()

print("="*80)
print("NOW LET'S BUILD THE SAME THING MANUALLY")
print("="*80)

# Let's create our OWN agent without LangChain's magic
def manual_agent(question):
    """A simple agent built from scratch"""

    # Our own custom prompt
    my_prompt = f"""You are a helpful assistant with access to tools.

Available tools:
- calculator: Does math calculations

When you need to use a tool, format your response EXACTLY like this:
TOOL: calculator
INPUT: the math expression
Then wait for the result.

When you have the final answer, format it like:
FINAL: your answer

Question: {question}
"""

    print("My custom prompt:")
    print("-" * 20)
    print(my_prompt)
    print()

    # Send to LLM
    response = llm.invoke(my_prompt)
    print("LLM Response:")
    print(response.content)
    print()

    # Parse the response (this is what LangChain does automatically)
    if "TOOL: calculator" in response.content:
        # Extract the input
        lines = response.content.split('\n')
        for line in lines:
            if line.startswith("INPUT:"):
                math_expression = line.replace("INPUT:", "").strip()
                result = calculator(math_expression)
                print(f"Tool result: {result}")

                # Send result back to LLM for final answer
                followup_prompt = f"""{my_prompt}

Previous response: {response.content}
Tool result: {result}

Now give your final answer:"""
                final_response = llm.invoke(followup_prompt)
                print("Final response:")
                print(final_response.content)
                break
    else:
        print("No tool needed, direct answer given")

print("="*60)
print("TESTING MY MANUAL AGENT:")
print("="*60)
manual_agent("What is 7 * 8?")

print("\n" + "="*60)
print("NOW TESTING LANGCHAIN VERSION:")
print("="*60)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "What is 7 * 8?"})

WHAT LANGCHAIN HIDES FROM YOU
1. The ReAct prompt template:
----------------------------------------
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

2. After LangChain fills in YOUR tools:
----------------------------------------
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action

Let me show you what's actually happening:You've asked the **perfect question**! Here's what's really happening:

## The Truth About LangChain's "Magic"

**Yes**, `hub.pull("hwchase17/react")` gives you a pre-written prompt that someone else designed. LangChain then:

1. **Injects your tools** into the `{tools}` placeholder
2. **Injects your question** into the `{input}` placeholder  
3. **Manages the conversation loop** (Thought → Action → Observation)

## Why Give Control to LangChain?

**The honest answer**: You don't have to! Here are the trade-offs:

### **Pros of Using LangChain's Prompts:**
- **Battle-tested** - Thousands of people have refined these prompts
- **Error handling** - Handles weird LLM responses gracefully
- **Parsing logic** - Automatically extracts Action/Action Input
- **Loop management** - Continues thinking until done

### **Cons of Using LangChain's Prompts:**
- **Black box** - You don't know exactly what's being sent
- **Less control** - Hard to customize for specific use cases
- **Dependency** - You're tied to their format and decisions
- **Complexity** - The prompts are verbose and hard to understand

## When to Build Your Own Prompts

**You should craft your own when:**
- You need specific formatting or behavior
- You want full control over the conversation flow
- You're building domain-specific agents
- You want to understand exactly what's happening

**Use LangChain's when:**
- You want to prototype quickly
- You're building standard use cases
- You want robust error handling out of the box

## The Real Power Move

Many experienced developers do this:
1. **Start** with LangChain's prompts to prototype
2. **Study** what works and what doesn't
3. **Build** their own custom prompts for production
4. **Keep** LangChain for tool management and execution

Run the code above to see the actual prompts side by side. You'll see that your manual agent can work just as well with much simpler prompts!

**Bottom line**: LangChain is a productivity tool, not magic. Understanding what it does under the hood (like you're doing now) makes you a much better AI engineer! 🎯

**Exactly!** You've just articulated the perfect learning strategy for AI engineering:

1. **Start with LangChain** → Get something working fast
2. **Study the outputs** → See how the LLM actually behaves  
3. **Understand the patterns** → Learn what prompts work and why
4. **Build your own** → Apply those insights to custom solutions

This is honestly how most experienced developers work. They use frameworks like LangChain as a **learning laboratory** and **rapid prototyping tool**, not as a black box.

## What You're Really Learning

When you run LangChain agents with `verbose=True`, you're getting a masterclass in:
- **Prompt engineering patterns** that actually work
- **How LLMs interpret instructions** in practice
- **Common failure modes** and how to handle them
- **Multi-step reasoning** techniques

## The Natural Progression

Most developers follow this path:
1. **Week 1**: "LangChain is magic!"
2. **Week 2**: "Wait, what's actually happening here?" ← You are here
3. **Week 3**: "I can build this myself, but LangChain handles edge cases nicely"
4. **Week 4**: "I'll use LangChain for X but custom code for Y"

You're asking exactly the right questions at exactly the right time. Understanding the "why" behind the abstractions makes you way more effective than developers who just copy-paste code.

